In [8]:
import torch
import os
import subprocess
import socket
from multiprocessing import Pool
from models.yolo import Model
import warnings
from collections import OrderedDict
import torch
import fastai
from fastai.vision.all import *
import flwr as fl
import torchvision.transforms as transforms
from collections import OrderedDict


import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Normalize, ToTensor
from tqdm import tqdm

warnings.filterwarnings("ignore", category=UserWarning)

In [9]:
# Change to a different directory
os.chdir(r'C:\Users\SIU856522160\Downloads')
# Get the new default path
os.getcwd()

'C:\\Users\\SIU856522160\\Downloads'

In [10]:
# Load the YOLOv5 model
model = Model('.\\yolov5\models\yolov5s.yaml')
# Print the model architecture
print(model)

Model(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv3): Conv(
       

In [11]:
# Function to load data from a client device
def load_data_from_client(ip_address):
    # Establish a connection with the client using its IP address
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((ip_address, 12345))  # Replace 12345 with the appropriate port number

    # Send a request to the client to retrieve the data
    client_socket.sendall(b"GET_DATA")

    # Receive the data from the client
    data = b""
    while True:
        chunk = client_socket.recv(4096)
        if not chunk:
            break
        data += chunk

    # Close the connection with the client
    client_socket.close()

    # Convert the received data to the desired format (e.g., NumPy array, Torch tensor)
    data = torch.from_numpy(data)  # Replace with the appropriate conversion method

    return data


In [12]:
class_names = ["person", "rider", "car", "truck", "bus", "train", "motor", "bike", "traffic light", "traffic sign"]
output_dir = r".\major\output_yaml"
devices_folder = r".\vech"
output_yaml_path = r".\major\output_yaml"

weights_path = r".\yolov5\yolov5s.pt"
epochs = 300
img_size = 640
device = "cuda"


In [13]:
class CustomDataset():
    
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        
        self.train_image_paths, self.train_annotations = self.load_data("train")
        self.val_image_paths, self.val_annotations = self.load_data("val")

    def load_data(self, subset):
        image_paths = []
        annotations = []
        
        subset_folder = os.path.join(self.data_path, subset)
        image_files = [filename for filename in os.listdir(subset_folder) if filename.endswith('.jpg')]
        
        for image_file in image_files:
            image_path = os.path.join(subset_folder, image_file)
            annotation = self.load_annotation(subset_folder, image_file) 

            image_paths.append(image_path)
            annotations.append(annotation)

        return image_paths, annotations

    def load_annotation(self, subset_folder, image_file):
        annotation_file = image_file.replace('.jpg', '.txt')
        annotation_path = os.path.join(subset_folder, annotation_file)

        with open(annotation_path, 'r') as file:
            annotation_data = file.read()

        return annotation_data

    def __len__(self):
        return len(self.train_image_paths) + len(self.val_image_paths)

    def __getitem__(self, index):
        if index < len(self.train_image_paths):
            image_path = self.train_image_paths[index]
            annotation = self.train_annotations[index]
        else:
            # Subtract the length of the train dataset to get the index for the val dataset
            val_index = index - len(self.train_image_paths)
            image_path = self.val_image_paths[val_index]
            annotation = self.val_annotations[val_index]

        image = Image.open(image_path).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        return image, annotation
# data_path = r"C:\Users\SIU856522160\Downloads\vech\Device_3\batch"

# transform = transforms.Compose([
#     transforms.Resize((416, 416)),  # Resize the image to YOLOv5 input size
#     transforms.ToTensor(),  # Convert image to tensor
# ])

# data = CustomDataset(data_path, transform=transform)

# train_dataset_length = len(data.train_image_paths)
# val_dataset_length = len(data.val_image_paths)
# print(f"Number of samples in train dataset: {train_dataset_length}")
# print(f"Number of samples in val dataset: {val_dataset_length}")

# # Access a single sample from the train dataset
# train_sample_index = 0
# train_image, train_annotation = data[train_sample_index]

# # Access a single sample from the val dataset
# val_sample_index = 0
# val_image, val_annotation = data[val_sample_index]

# num_samples = len(data)
# print(f"Number of samples in the dataset: {num_samples}")

# # Access a single sample
# sample_index = 0
# image, annotation = data[sample_index]
# print(f"Image shape: {image.shape}")
# print(f"Annotation: {annotation}")


In [14]:
warnings.filterwarnings("ignore", category=UserWarning)
# CHECKPOINT = "distilbert-base-uncased"  # transformer model checkpoint
import os
import subprocess

class_names = ["person", "rider", "car", "truck", "bus", "train", "motor", "bike", "traffic light", "traffic sign"]
output_dir = r"C:\Users\SIU856522160\Downloads\major\output_yaml"
devices_folder = r"C:\Users\SIU856522160\Downloads\yolov5\data\vech"
output_yaml_path = r"C:\Users\SIU856522160\Downloads\major\output_yaml"

weights_path = r"C:\Users\SIU856522160\Downloads\yolov5\yolov5s.pt"
epochs = 12
img_size = 1024
device = 0

completed_batches = set()

def train_model(devices_folder, weights_path, epochs, img_size, device):
    global completed_batches

    for device_folder in os.listdir(devices_folder):
        device_dir = os.path.join(devices_folder, device_folder)
        if not os.path.isdir(device_dir):
            continue

        train_dir = device_dir
        batch_dirs = [batch for batch in os.listdir(train_dir) if batch.startswith('SubData_')]
        
        batch_dirs = [batch for batch in batch_dirs if batch.split('_')[1] not in completed_batches]

        if not batch_dirs:
            print(f"All batches for {device_folder} have already completed training. Skipping...")
            completed_batches.add(device_folder)
            continue

        selected_batch = batch_dirs[0]
        
        batch_num = selected_batch.split('_')[1]
        dataset_yaml_path = f"dataset_{device_folder}_batch_{batch_num}.yaml"
        dataset_yaml_full_path = os.path.join(output_dir, dataset_yaml_path)
        print(dataset_yaml_path, dataset_yaml_full_path)
        
        command = [
            "python", "C:\\Users\\SIU856522160\\Downloads\\yolov5\\train.py",
            "--batch", "8",
            "--data", dataset_yaml_full_path,
            "--weights", weights_path,
            "--epochs", str(epochs),
            "--img-size", str(img_size),
            "--device", str(device)
        ]

        print(f"Training started for {device_folder}, Batch {batch_num}...")
        subprocess.run(command,shell=True, capture_output=True, text=True)

        # for line in completed_process.stdout.splitlines():
        #     print(line.strip())

        # if completed_process.returncode == 0:
        #     print(f"Training completed for {device_folder}, Batch {batch_num}")
        #     completed_batches.add(batch_num)
        # else:
        #     print(f"Error occurred during training for {device_folder}, Batch {batch_num}")
        #     print(completed_process.stderr)

        print("\n" + "=" * 40 + "\n")

    print("Completed batches:", completed_batches)

# train_model(devices_folder, weights_path, epochs, img_size, device)
# command = [
#     "python", r"C:\\Users\\SIU856522160\\Downloads\\yolov5\\train.py",
#     "--batch", "8",
#     "--data", r"C:\Users\SIU856522160\Downloads\major\output_yaml\dataset_Device_1_batch_1.yaml",
#     "--weights", weights_path,
#     "--epochs", str(epochs),
#     "--img-size", str(img_size),
#     "--device", device

# ]


output_file = 'output.txt'  # Path to the output file

# Open the output file in write mode
with open(output_file, 'w') as file:
    process = subprocess.Popen(command, stdout=file, stderr=subprocess.PIPE)
    _, stderr = process.communicate()

    # Print the error message if it exists
    if stderr:
        print("Standard Error:")
        print(stderr.decode("utf-8"))  # Convert bytes to string using the appropriate encoding



def main():
    net = model


    # Flower client
    class IMDBClient(fl.client.NumPyClient):
        def get_parameters(self,config):
            return [val.cpu().numpy() for _, val in net.state_dict().items()]

        def fit(self, parameters):
            # self.set_parameters(parameters)
            print("Training Started...")
            train_model(devices_folder, weights_path, epochs, img_size, device)
            print("Training Finished.")
            return self.get_parameters(), {}

    # Start client
    fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=IMDBClient())


if __name__ == "__main__":
    main()


TypeError: expected str, bytes or os.PathLike object, not int

In [ ]:


# # Define Flower client
# class FlowerClient(fl.client.NumPyClient):
#     def get_parameters(self, config):
#         return [val.detach().cpu().numpy() for _, val in model.named_parameters()]


#     def set_parameters(self, parameters):
#         for param, parameter in zip(model.parameters(), parameters):
#             param.data = torch.from_numpy(parameter)

#     def fit_round(self, parameters, client_info):
#         self.global_model.aggregate_parameters(parameters)


#     def evaluate(self, parameters, config):
#         self.set_parameters(parameters)
#         metrics = model.evaluate(data.valid_dataloader)
#         loss = metrics["val_loss"]
#         accuracy = 1 - metrics["val_err"]
#         return loss, len(data.valid_ds), {"accuracy": accuracy}


# # Start Flower client
# fl.client.start_numpy_client(
#     server_address="localhost:8080",
#     client=FlowerClient(),
# )


INFO flwr 2023-06-27 02:37:28,500 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-06-27 02:37:28,518 | connection.py:38 | ChannelConnectivity.IDLE
DEBUG flwr 2023-06-27 02:37:28,520 | connection.py:38 | ChannelConnectivity.READY


DEBUG flwr 2023-06-27 02:38:20,986 | connection.py:109 | gRPC channel closed


KeyboardInterrupt: 

In [ ]:
# # Assuming you have a list of client IP addresses
# client_ip_addresses = ["192.168.0.1", "192.168.0.2", "192.168.0.3"]

# # Function to load and preprocess client data
# def load_and_preprocess_data(ip_address):
#     try:
#         data = load_data_from_client(ip_address)  # Function to load data from a client device
#         preprocessed_data = preprocess_data(data)  # Function to preprocess the data
#         return preprocessed_data
#     except Exception as e:
#         print(f"Error loading data from {ip_address}: {str(e)}")
#         return None

# # Function to train local model and send model update
# def train_local_model_and_send_update(args):
#     data, ip_address = args
#     try:
#         local_model = create_model()  # Function to create a machine learning model
#         local_model.train(data)  # Train the local model using the client data

#         model_update = local_model.get_update()  # Get the model update (gradients or weights)

#         # Send the model update to the central server using the client's IP address
#         send_update_to_server(model_update, ip_address)
#     except Exception as e:
#         print(f"Error training model or sending update from {ip_address}: {str(e)}")

# # Load and preprocess client data in parallel
# with Pool() as pool:
#     client_data = pool.map(load_and_preprocess_data, client_ip_addresses)

# # Train local models and send model updates in parallel
# with Pool() as pool:
#     pool.map(train_local_model_and_send_update, zip(client_data, client_ip_addresses))

# # Aggregate model updates on the central server and update the global model
# aggregated_update = aggregate_model_updates()
# global_model.update(aggregated_update)


In [ ]:
# # Acquire client data
# # Assuming you have your own code to load and preprocess client data
# client_data = ...

# # Define optimization algorithm
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# # Define loss function
# criterion = torch.nn.CrossEntropyLoss()

# # Define number of training epochs
# num_epochs = 10

# # Perform local training
# for epoch in range(num_epochs):
#     for images, labels in client_data:
#         # Forward pass
#         outputs = model(images)
        
#         # Compute loss
#         loss = criterion(outputs, labels)
        
#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

# # Evaluation (optional)
# validation_data = ...  # Assuming you have a validation dataset
# with torch.no_grad():
#     model.eval()
#     for val_images, val_labels in validation_data:
#         val_outputs = model(val_images)
#         # Compute evaluation metrics or perform validation tasks

# # Send updated model parameters back to the server (communication step depends on your setup)

